In [11]:
path = "malimg_paper_dataset_imgs"

In [12]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model

# Definir el autoencoder
input_img = Input(shape=(64, 64, 3))

# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Construir el modelo de autoencoder
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Resumen del modelo
autoencoder.summary()


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 16, 16, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_6 (UpSampling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_7 (UpSampling2D)  │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 64, 64, 3)      │           867 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,507 (115.26 KB)

 Trainable params: 29,507 (115.26 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Configurar el generador de datos para el entrenamiento del autoencoder
data = ImageDataGenerator(rescale=1./255)
train_generator = data.flow_from_directory(
    path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='input', 
    shuffle=True
)

# Entrenar el autoencoder con la pérdida de error cuadrático medio (mean squared error)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el autoencoder
autoencoder.fit(train_generator, epochs=10, steps_per_epoch=len(train_generator))

# Guardar el modelo del autoencoder
autoencoder.save('autoencoder.h5')

Found 9339 images belonging to 25 classes.
Epoch 1/10


c:\Users\Fernando Lopez\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


292/292 ━━━━━━━━━━━━━━━━━━━━ 44s 144ms/step - loss: 0.0895
Epoch 2/10
292/292 ━━━━━━━━━━━━━━━━━━━━ 0s 27us/step - loss: 0.0000e+00
Epoch 3/10


c:\Users\Fernando Lopez\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


292/292 ━━━━━━━━━━━━━━━━━━━━ 24s 79ms/step - loss: 0.0744
Epoch 4/10
292/292 ━━━━━━━━━━━━━━━━━━━━ 0s 17us/step - loss: 0.0000e+00
Epoch 5/10
292/292 ━━━━━━━━━━━━━━━━━━━━ 24s 81ms/step - loss: 0.0661
Epoch 6/10
292/292 ━━━━━━━━━━━━━━━━━━━━ 0s 14us/step - loss: 0.0000e+00
Epoch 7/10
292/292 ━━━━━━━━━━━━━━━━━━━━ 24s 81ms/step - loss: 0.0621
Epoch 8/10
292/292 ━━━━━━━━━━━━━━━━━━━━ 0s 14us/step - loss: 0.0000e+00
Epoch 9/10
292/292 ━━━━━━━━━━━━━━━━━━━━ 24s 81ms/step - loss: 0.0586
Epoch 10/10
292/292 ━━━━━━━━━━━━━━━━━━━━ 0s 14us/step - loss: 0.0000e+00
